# Introduction

This notebook presents implementations of an **Article Writer app using a multi-agent approach** with two agents executors:

- **Researcher**: An agent that queries the DuckDuckGo search API and is useful for when you need to answer questions about current events.

- **Writer**: An agent that writes text by leveraging an LLM and is useful for when you need to write an engaging text based on a set of insights.

The code is created using the [crewAI](https://github.com/crewAIInc/crewAI) library, which is a cutting-edge framework for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks.

<p align="center">
  <img src="img/crewai.png" alt="" width="400">
</p>

<center>Image source: Github - CrewAI.</center>

**By the end of the notebook, it is discussed the hierarchical structure nomenclature between CrewAi and [LangGraph](https://www.langchain.com/langgraph)**, which is a framework for building stateful, multi-actor agents with LLMs that can handle complex scenarios and collaborate with humans.

In [ ]:
#! pip install --quiet \
#              crewai==0.41.1 \
#              duckduckgo-search==6.3.0 \
#              langchain-community==0.2.10 \
#              langchain-openai==0.1.20 \
#              poetry==1.8.4 \
#              python-dotenv==1.0.1

In [ ]:
import os

from crewai import Agent, Task, Crew, Process
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun

In [ ]:
use_google_drive_api = False
google_drive_path = 'gdrive/My Drive/Profissional/Haleon/Talks/Multi Agent vs Single Agent with LLM-based Tools/code'

if(use_google_drive_api):
  from google.colab import drive
  drive.mount('/content/gdrive/', force_remount=True)
  # Changing the current working directory to the notebook folder
  os.chdir(google_drive_path)
  print(os.getcwd())

In [ ]:
# OpenAI API Key

_ = load_dotenv(find_dotenv()) # read local .env file
api_key = os.environ["OPENAI_API_KEY"]

# Article Writer with a Predefined Sequential Process

First, we will start by creating an Article Writer app that has a predefined sequential process:
1. Task: Search about AI. Assigned: Researcher
2. Task: Write an article. Assigned: Writer.

<p align="center">
  <img src="img/article_writer_via_multi_agent_sequential_without_delegation_using_crewai.png" alt="" width="300">
</p>

<center>Image source: Author.</center>

In [ ]:
# Instantiate an LLM

llm = ChatOpenAI(
    api_key=api_key,
    temperature=0.4,
    model_name="gpt-4o",
)

In [ ]:
# Instantiate the agent tools

# The research tool

search_tool = DuckDuckGoSearchRun()

In [ ]:
type(search_tool)

langchain_community.tools.ddg_search.tool.DuckDuckGoSearchRun

In [ ]:
# Define your agents with roles and goals

researcher = Agent(
    role='Senior Research Analyst',
    goal='Uncover cutting-edge developments in AI and data science',
    backstory="""You work at a leading tech think tank.
    Your expertise lies in identifying emerging trends.
    You have a knack for dissecting complex data and presenting actionable insights.""",
    verbose=True,
    allow_delegation=False,
    tools=[search_tool],
    llm = llm
)

writer = Agent(
    role='Tech Content Strategist',
    goal='Craft compelling content on tech advancements',
    backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
    You transform complex concepts into compelling narratives.""",
    verbose=True,
    allow_delegation=False,
    llm = llm
)

In [ ]:
# Create tasks for your agents

task1 = Task(
    description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
    Identify key trends, breakthrough technologies, and potential industry impacts.""",
    expected_output="Full analysis report in bullet points",
    agent=researcher
)

task2 = Task(
    description="""Using the insights provided, develop an engaging blog
    post that highlights the most significant AI advancements.
    Your post should be informative yet accessible, catering to a tech-savvy audience.
    Make it sound cool, avoid complex words so it doesn't sound like AI.""",
    expected_output="Full blog post of at least 4 paragraphs",
    agent=writer
)

In [ ]:
# Instantiate your crew with a sequential process

crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    verbose=2, # You can set it to 1 or 2 to different logging levels
    process=Process.sequential,
)

In [ ]:
# Invoke the crew

result = crew.kickoff()
print("-----------------------------")
print(result)

 [2024-10-14 22:18:43][DEBUG]: == Working Agent: Senior Research Analyst
 [2024-10-14 22:18:43][INFO]: == Starting Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.
    Identify key trends, breakthrough technologies, and potential industry impacts.


> Entering new CrewAgentExecutor chain...
To conduct a comprehensive analysis of the latest advancements in AI in 2024, I will use the duckduckgo_search tool to gather information on key trends, breakthrough technologies, and potential industry impacts. 

Action: duckduckgo_search
Action Input: {"query": "latest advancements in AI 2024 trends breakthrough technologies industry impact"} 

10 Breakthrough Technologies. Every year, the reporters and editors at MIT Technology Review survey the tech landscape and pick 10 technologies that we think have the greatest potential to change ... In 2024, generative AI might actually become useful for the regular, non-tech person, and we are going to see more people tinke

## Allow Agent Interaction

Agents can interact with each other using crewAI's built-in delegation and communication mechanisms. This allows for dynamic task management and problem-solving within the crew, ensuring that each task is handled by the most suitable agent. For for info, see Allow Delegation [Agent Attributes](https://docs.crewai.com/core-concepts/Agents/#agent-attributes).

To illustrate the agent delegation feature in action, we will use only Task 2 in order to force the Writer Agent to delegate the research task to the Researcher Agent.

<p align="center">
  <img src="img/article_writer_via_multi_agent_sequential_with_delegation_using_crewai.png" alt="" width="300">
</p>

<center>Image source: Author.</center>

In [ ]:
# Include only the second task, but allow the writer agent to delete tasks

# Change initial setup to allow delegation
researcher.allow_delegation=True
writer.allow_delegation=True

crew = Crew(
    agents=[researcher, writer],
    tasks=[task2],
    verbose=2,
    process=Process.sequential,
)

result = crew.kickoff()
print("-----------------------------")
print(result)

 [2024-10-14 22:26:46][DEBUG]: == Working Agent: Crew Manager
 [2024-10-14 22:26:46][INFO]: == Starting Task: Using the insights provided, develop an engaging blog
    post that highlights the most significant AI advancements.
    Your post should be informative yet accessible, catering to a tech-savvy audience.
    Make it sound cool, avoid complex words so it doesn't sound like AI.


> Entering new CrewAgentExecutor chain...
To identify the most significant AI advancements recently, I should perform a search to gather current information on this topic. 

Action: duckduckgo_search
Action Input: {"query": "recent significant AI advancements 2023"} 

Some of the most profound implications for AI are in the field of bioscience and genetics, and in 2023, Google made a significant leap forward with its DeepMind-developed AlphaMissense model. A recap of the AI advances, challenges, and debates in 2023, covering open source, generative AI, employment, regulation, and industry verticals. Lear

## Hierarchical Mangagement


[Hierarchical Process](https://docs.crewai.com/core-concepts/Processes/#hierarchical-process) emulates a corporate hierarchy, CrewAI allows specifying a custom manager agent or automatically creates one, requiring the specification of a manager language model (manager_llm). This agent oversees task execution, including planning, delegation, and validation. Tasks are not pre-assigned; the manager allocates tasks to agents based on their capabilities, reviews outputs, and assesses task completion.

To illustrate the flexibility and the capabilities of having an LLM managing the process, we use unassigned tasks in order to force the LLM manager to allocate the tasks to agents, resulting the following diagram.

<p align="center">
  <img src="img/article_writer_via_multi_agent_hierarchical_with_delegation_using_crewai.png" alt="" width="300">
</p>

<center>Image source: Author.</center>

In [ ]:
# Example: Creating a crew with a hierarchical process
# Ensure to provide a manager_llm or manager_agent

# Change the initial setup to unassign an agent responsible for the task
# In order to force the attribution for the manager
# For more information, see https://docs.crewai.com/core-concepts/Tasks/#task-attributes
task1.agent = None
task2.agent = None

my_agents = [writer, researcher]
my_tasks = [task1, task2]

crew = Crew(
    agents=my_agents,
    tasks=my_tasks,
    process=Process.hierarchical,
    manager_llm=llm
    # or
    # manager_agent=my_manager_agent
)

result = crew.kickoff()
print("-----------------------------")
print(result)



> Entering new CrewAgentExecutor chain...
To identify key trends and breakthrough technologies in AI in 2024, I should conduct a search using the duckduckgo_search tool to gather information from recent publications, conferences, and expert opinions in the field of AI.

Action: duckduckgo_search
Action Input: {"query": "AI trends and breakthrough technologies 2024"} 

Artificial intelligence. AI for everything: 10 Breakthrough Technologies 2024. Generative AI tools like ChatGPT reached mass adoption in record time, and reset the course of an entire industry. By . 10 Breakthrough Technologies. Every year, the reporters and editors at MIT Technology Review survey the tech landscape and pick 10 technologies that we think have the greatest potential to change ... In 2024, generative AI might actually become useful for the regular, non-tech person, and we are going to see more people tinkering with a million little AI models. State-of-the-art AI models ... Here are the top 10 AI trends th

# Discussion: Is it really a hierarchical relationship?

This process is called hierarchical because an LLM or Agent called the manager who assigns tasks to the Agent executors. However, if you check LangGraph, for instance, this implementation is called Agent Supervisor since it introduces the concept of a single manager/supervisor node to route work between different worker agents/nodes, as shown in the figure below:

[Article on how to embed Google Drive images in Colab]: <> (*https://medium.com/@nanaliu/a-seamless-guide-to-embedding-photos-into-google-colab-894167bfde3a#:~:text=To%20insert%20the%20image%2C%20you%20can%20use%20the,in%20a%20Markdown%20cell%20in%20your%20Colab%20notebook%3A*)

<p align="center">
  <img src="img/supervisor-diagram.png" alt="" width="600">
</p>

<center>Image source: LangGraph - Agent Supervisor.</center>

But what if the job for a single worker becomes too complex? What if the number of workers becomes too large? For some applications, the system may be more effective if work is distributed hierarchically; that is, an agent assigns a task to another agent who will do the same in a hierarchical manner. You can do this by composing different subgraphs and creating a top-level supervisor, along with mid-level supervisors, as shown below.

<p align="center">
  <img src="img/hierarchical-diagram.png" alt="" width="600">
</p>

<center>Image source: LangGraph - Hierarchical Agent Teams.</center>

Sources:
1. [LangGraph - Agent Supervisor](https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/agent_supervisor.ipynb)
2. [LangGraph - Hierarchical Agent Teams](https://github.com/langchain-ai/langgraph/blob/a16a33806c60e80be1fb64b54f2e5835f6080c01/examples/multi_agent/hierarchical_agent_teams.ipynb)